In [53]:
import torch
import torch.nn
import numpy as np

a = np.random.randint(0, 10, 5)
a = torch.tensor(a)
a

tensor([0, 3, 0, 9, 4], dtype=torch.int32)

In [54]:
b = torch.tensor(np.random.randint(0, 10, 5))
b

tensor([3, 9, 7, 4, 2], dtype=torch.int32)

In [55]:
c = torch.add(a, b)
c_summed = torch.sum(c)

c, c_summed

(tensor([ 3, 12,  7, 13,  6], dtype=torch.int32), tensor(41))

In [56]:
d = torch.tanh(b.float())
d

tensor([0.9951, 1.0000, 1.0000, 0.9993, 0.9640])

In [57]:
torch.mean(d)

tensor(0.9917)

In [78]:
import tensorflow as tf
import numpy as np

f = tf.constant(np.random.randint(0, 10, 5), shape=(1, 5), dtype=tf.int32)
print(f)
g = tf.constant(np.random.randint(0, 10, 5), shape=(1, 5), dtype=tf.int32)
print(g)
result = A + B
tanh = tf.tanh(tf.to_float(result))


with tf.Session() as sess:
    print("f: {}    g: {}\n".format(sess.run(f), sess.run(g)))
    print("f + g: {}\n".format(sess.run(result)))
    print("tanh(f+g): {}".format(sess.run(tanh)))

Tensor("Const_37:0", shape=(1, 5), dtype=int32)
Tensor("Const_38:0", shape=(1, 5), dtype=int32)
f: [[5 6 9 7 0]]    g: [[3 9 4 8 9]]

f + g: [[14 16  6  5 16]]

tanh(f+g): [[1.         1.         0.99998784 0.99990916 1.        ]]


In [79]:
### Example on MNIST

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn.functional as F
import numpy as np

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dense1 = nn.Linear(12*12*64, 128)
        self.dense2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.dropout(x, 0.25)
        x = x.view(-1, 12*12*64)
        x = F.relu(self.dense1(x))
        x = F.dropout(x, 0.5)
        x = self.dense2(x)
        return F.log_softmax(x, dim=1)



In [3]:
def train(model, device, train_loader, criterion, optimizer, epoch, save_dir='model.ckpt'):
    total_step = len(train_loader)
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            
    torch.save(model.state_dict(), 'pytorch_mnist_cnn.ckpt')

In [4]:
from sklearn.metrics import roc_auc_score

def test(model, device, test_loader):
    
    preds = []
    y_true = []
    # Test the model
    model.eval()  # Set model to evaluation mode.
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()    
            detached_pred = predicted.detach().cpu().numpy()
            detached_label = labels.detach().cpu().numpy()
            for f in range(0, len(detached_pred)):
                preds.append(detached_pred[f])
                y_true.append(detached_label[f])

        print('Test Accuracy of the model on the 10000 test images: {:.2%}'.format(correct / total))

        preds = np.eye(num_classes)[preds]
        y_true = np.eye(num_classes)[y_true]    
        auc = roc_auc_score(preds, y_true)
        print("AUC: {:.2%}".format (auc))
        

In [5]:
#### Hyperparameters
num_epochs = 15
num_classes = 10
batch_size = 128
learning_rate = 0.001

# Load MNIST data set
train_dataset = torchvision.datasets.MNIST(root='../../data/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../data/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)



model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


## Training phase

for epoch in range(0, num_epochs):
    train(model, device, train_loader, criterion, optimizer, epoch)
    
## Testing phase


test(model, device, test_loader)

torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])

torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])

torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])
torch.Size([128, 1, 28, 28])
torch.Size([128, 32, 26, 26])
torch.Size([128, 64, 24, 24])
torch.Size([128, 64, 12, 12])
torch.Size([128, 9216])

KeyboardInterrupt: 

In [ ]:
## sequential
import torch.nn as nn

model = nn.Sequential(
            nn.Conv2d(1, 32, 3, 1),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, 1),
            nn.Sigmoid()
            )

## ModuleList

class ModuleListModel(nn.Module):
    def __init__(self):
        super(ModuleListModel, self).__init__()
        self.conv_1 = nn.Conv2d(1, 32, 3, 1)
        self.conv_2 = nn.Conv2d(32, 64, 3, 1)
        self.dense_1 = nn.Linear(64*64, 128)
        self.output = nn.Linear(128, n_classes)
        
        
    def forward(self, x):
        x = nn.functional.relu(self.conv_1(x))
        x = nn.functional.relu(self.conv_2(x))
        x = nn.functional.max_pool2d(x, 2, 2)
        x = nn.functional.dropout(x, 0.25)
        x = x.view(-1, 64*64)
        x = nn.functional.reu(self.dense_1(x))
        x = nn.functional.dropout(x, 0.5)
        x = nn.functional.log_softmax(self.output(x), dim=1)
        return x
    

model = ModuleListModel().to(device)

In [ ]:
def forward(self, x):
    x = nn.functional.relu(self.conv_1(x))
    x = nn.functional.dropout(x, 0.25)
    x = nn.functional.relu(self.conv_2(x))
    ...

In [ ]:
def forward(self, x):
    ...
    x = nn.functional.dropout(x, 0.5)
    x = nn.functional.softmax(self.dense_1(x), dim=1)
    return x
    

In [ ]:
def forward(self, x):
    ...
    x = nn.functional.dropout(x, 0.5)
    x = nn.functional.log_softmax(self.dense_1(x), dim=1)
    return x

In [ ]:
def forward(self, x):
    ...
    x = nn.functional.dropout(x, 0.5)
    x = nn.functional.sigmoid(self.dense_1(x), dim=1)
    return x